In [93]:
import src.functions_search as fs
import src.functions_rank as fr
import src.functions_mongo as fm
import googlemaps
import geopy.distance
from pymongo import MongoClient
from datetime import datetime
import os
import requests
import json
import time
import pandas as pd
import folium
from dotenv import load_dotenv
load_dotenv()

True

In [97]:
airports=pd.read_csv("./output/filtered_airports.csv")
offices=pd.read_csv("./output/filtered_offices.csv")

In [98]:
print(len(offices),len(airports))
display(offices)

85 50


,Unnamed: 0,name,category,city,country,latitude,longitude,is_gaming,total_companies
0,0,Thoof,web,Austin,USA,30.268735,-97.745209,0,36
1,1,RockYou,games_video,Redwood City,USA,37.484619,-122.206893,1,23
2,2,Carbonite,network_hosting,Boston,USA,42.350571,-71.076869,0,26
3,3,oDesk,web,Redwood City,USA,37.479952,-122.178557,0,23
4,4,Bazaarvoice,software,Austin,USA,30.407545,-97.717667,0,36
...,...,...,...,...,...,...,...,...,...
80,80,Virdia,cleantech,Redwood City,USA,37.523311,-122.259154,0,23
81,81,Buzzient,enterprise,Boston,USA,42.362856,-71.083762,0,26
82,82,Nuventix,cleantech,Austin,USA,30.239229,-97.834211,0,36
83,83,Pearl Therapeutics,biotech,Redwood City,USA,37.502169,-122.217727,0,23


In [99]:
print(offices.iloc[0])
print(airports.iloc[0])
print(closest_airport(offices.iloc[0],airports))

Unnamed: 0               0
name                 Thoof
category               web
city                Austin
country                USA
latitude           30.2687
longitude         -97.7452
is_gaming                0
total_companies         36
Name: 0, dtype: object
Unnamed: 0                                        0
code                                            ABQ
lat                                         35.0494
lon                                        -106.625
name              Albuquerque International Airport
city                                    Albuquerque
state                                    New Mexico
country                               United States
woeid                                      12518564
icao                                           KABQ
direct_flights                                   41
carriers                                         25
Name: 0, dtype: object
10.618504347609518


In [100]:
dist_airport=[]
for x in range(len(offices)):
    office=offices.iloc[x]
    dist_airport.append(fs.closest_airport(office,airports))
print(len(dist_airport))

85


In [101]:
offices['dist_airport'] = dist_airport 

In [102]:
display(offices.head())

,Unnamed: 0,name,category,city,country,latitude,longitude,is_gaming,total_companies,dist_airport
0,0,Thoof,web,Austin,USA,30.268735,-97.745209,0,36,10.618504
1,1,RockYou,games_video,Redwood City,USA,37.484619,-122.206893,1,23,21.824782
2,2,Carbonite,network_hosting,Boston,USA,42.350571,-71.076869,0,26,4.652931
3,3,oDesk,web,Redwood City,USA,37.479952,-122.178557,0,23,24.078317
4,4,Bazaarvoice,software,Austin,USA,30.407545,-97.717667,0,36,23.260582


In [103]:
##Function to find closest airport                 
def closest_starbucks(office):
    coords_off = (office.get('latitude'),office.get('longitude'))
    coords_off2 = ("{},{}".format(coords_off[0],coords_off[1]))
    nbs_list=fs.nearbysearchName("starbucks",coords_off2,1000)
    dist = []
    a=0
    if nbs_list==[]:
        dist.append(a)
    else:
        starbucks=pd.DataFrame(fs.getLocation(nbs_list)).iloc[0]
        coords_str=(starbucks.get('latitude'),starbucks.get('longitude'))
        geo_dist=(geopy.distance.geodesic(coords_off, coords_str).m)
        dist.append(geo_dist)
        return dist[0]


In [104]:
dist_starbucks=[]
for x in range(len(offices)):
    office=offices.iloc[x]
    dist_starbucks.append(fs.closest_starbucks(office))

offices['dist_starbucks'] = dist_starbucks

display(offices)

,Unnamed: 0,name,category,city,country,latitude,longitude,is_gaming,total_companies,dist_airport,dist_starbucks
0,0,Thoof,web,Austin,USA,30.268735,-97.745209,0,36,10.618504,223.125453
1,1,RockYou,games_video,Redwood City,USA,37.484619,-122.206893,1,23,21.824782,1392.763284
2,2,Carbonite,network_hosting,Boston,USA,42.350571,-71.076869,0,26,4.652931,360.800203
3,3,oDesk,web,Redwood City,USA,37.479952,-122.178557,0,23,24.078317,682.174891
4,4,Bazaarvoice,software,Austin,USA,30.407545,-97.717667,0,36,23.260582,1040.105325
...,...,...,...,...,...,...,...,...,...,...,...
80,80,Virdia,cleantech,Redwood City,USA,37.523311,-122.259154,0,23,15.519179,658.716597
81,81,Buzzient,enterprise,Boston,USA,42.362856,-71.083762,0,26,4.689089,354.447183
82,82,Nuventix,cleantech,Austin,USA,30.239229,-97.834211,0,36,16.757988,810.101295
83,83,Pearl Therapeutics,biotech,Redwood City,USA,37.502169,-122.217727,0,23,19.833559,NaN


In [105]:
print(offices.dtypes)

Unnamed: 0           int64
name                object
category            object
city                object
country             object
latitude           float64
longitude          float64
is_gaming            int64
total_companies      int64
dist_airport       float64
dist_starbucks     float64
dtype: object


In [106]:
##Function to find closest whatever place we want with nearbyserach gmaps query
def closest_gtype(office,query,radius,gtype):
    coords_off = (office.get('latitude'),office.get('longitude'))
    coords_off2 = ("{},{}".format(coords_off[0],coords_off[1]))
    nbs_list=fs.nearbysearchType(query,coords_off2,radius,gtype)
    dist = []
    a=0
    if nbs_list==[]:
        dist.append(a)
    else:
        df=pd.DataFrame(fs.getLocation(nbs_list)).iloc[0]
        coords_df=(df.get('latitude'),df.get('longitude'))
        geo_dist=(geopy.distance.geodesic(coords_off, coords_df).m)
        dist.append(geo_dist)
    return dist[0]

In [107]:
## 4. Distance to Closest Vegan Restaurant
dist_vegan=[]
for x in range(len(offices)):
    office=offices.iloc[x]
    dist_vegan.append(fs.closest_gtype(office,"vegan",500,"restaurant"))

offices['dist_vegan'] = dist_vegan

In [ ]:
## 5. Distance to Closest Waldorf school
dist_school=[]
for x in range(len(offices)):
    office=offices.iloc[x]
    dist_school.append(fs.closest_gtype(office,"waldorf",4000,"school"))

offices['dist_school'] = dist_school

In [81]:
display(offices)

,Unnamed: 0,name,category,city,country,latitude,longitude,is_gaming,total_companies,dist_airport,dist_starbucks,dist_vegan,dist_school
0,0,Thoof,web,Austin,USA,30.268735,-97.745209,0,36,10.618504,223.125453,291.990161,2139.160885
1,1,RockYou,games_video,Redwood City,USA,37.484619,-122.206893,1,23,21.824782,NaN,388.944091,3390.832648
2,2,Carbonite,network_hosting,Boston,USA,42.350571,-71.076869,0,26,4.652931,360.800203,177.114680,1132.796641
3,3,oDesk,web,Redwood City,USA,37.479952,-122.178557,0,23,24.078317,682.174891,133.086962,2897.807838
4,4,Bazaarvoice,software,Austin,USA,30.407545,-97.717667,0,36,23.260582,636.012119,421.344748,1018.059403
...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,80,Virdia,cleantech,Redwood City,USA,37.523311,-122.259154,0,23,15.519179,658.716597,466.455844,4695.307455
81,81,Buzzient,enterprise,Boston,USA,42.362856,-71.083762,0,26,4.689089,354.447183,295.946630,1867.704347
82,82,Nuventix,cleantech,Austin,USA,30.239229,-97.834211,0,36,16.757988,810.101295,236.683629,7526.674891
83,83,Pearl Therapeutics,biotech,Redwood City,USA,37.502169,-122.217727,0,23,19.833559,NaN,0.000000,5558.182031


In [90]:
##6. Ranking based on functions_rank
offices['rank'] = offices.apply (lambda row: fr.rank_airport(row), axis=1) + offices.apply (lambda row: fr.rank_starbucks(row), axis=1) + offices.apply (lambda row: fr.rank_vegan(row), axis=1) + offices.apply (lambda row: fr.rank_schools(row), axis=1)
offices=offices.sort_values('rank',ascending=False)
display(offices.head(20))
print(offices.dtypes)

,Unnamed: 0,name,category,city,country,latitude,longitude,is_gaming,total_companies,dist_airport,dist_starbucks,dist_vegan,dist_school,rank,rank_Airport,rank_Starbucks,rank_Vegan,rank_Schools
73,73,Shareaholic,web,Boston,USA,42.364756,-71.103259,0,26,6.238827,104.974538,80.897534,2312.798032,25,10,10,10,10
6,6,Cloudant,enterprise,Boston,USA,42.358920,-71.057810,0,26,2.829292,175.923061,153.701668,2950.272556,20,10,10,10,10
68,68,SCVNGR,games_video,Boston,USA,42.340653,-71.068003,1,26,4.764528,593.158218,40.895928,9585.965133,20,10,10,10,10
35,35,Optaros,consulting,Boston,USA,42.356812,-71.058185,0,26,2.979413,132.543216,174.009406,2820.901034,20,10,10,10,10
2,2,Carbonite,network_hosting,Boston,USA,42.350571,-71.076869,0,26,4.652931,360.800203,177.114680,1132.796641,19,10,10,10,10
16,16,Intronis,enterprise,Boston,USA,42.351105,-71.049124,0,26,2.869366,329.475906,102.922434,3366.732805,17,10,10,10,10
17,17,Qik,mobile,Redwood City,USA,37.486368,-122.230116,0,23,20.194579,15.727920,156.383293,4713.608722,17,2,10,10,10
67,67,Tatto Media,advertising,Boston,USA,42.353491,-71.056465,0,26,3.090434,105.232815,265.314992,2830.020365,17,10,10,10,10
62,62,WordStream,advertising,Boston,USA,42.354257,-71.056605,0,26,3.044372,143.974665,418.268080,2843.147698,17,10,10,10,10
40,40,Jumptap,mobile,Boston,USA,42.368633,-71.075305,0,26,3.903119,146.335896,331.349684,2713.999533,17,10,10,10,10


Unnamed: 0           int64
name                object
category            object
city                object
country             object
latitude           float64
longitude          float64
is_gaming            int64
total_companies      int64
dist_airport       float64
dist_starbucks     float64
dist_vegan         float64
dist_school        float64
rank                 int64
rank_Airport         int64
rank_Starbucks       int64
rank_Vegan           int64
rank_Schools         int64
dtype: object


In [91]:
##An the winner is: 
print(offices.iloc[0])

Unnamed: 0                  73
name               Shareaholic
category                   web
city                    Boston
country                    USA
latitude               42.3648
longitude             -71.1033
is_gaming                    0
total_companies             26
dist_airport           6.23883
dist_starbucks         104.975
dist_vegan             80.8975
dist_school             2312.8
rank                        25
rank_Airport                10
rank_Starbucks              10
rank_Vegan                  10
rank_Schools                10
Name: 73, dtype: object
